# Installing/Importing Python Libraries

In [ ]:
# 1. Install the necessary libraries if you don't have them
# !pip install pandas
# !pip install numpy
# !pip install sklearn
# !pip install matplotlib

# 2. Import the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# 3. Import specific modules from the libraries
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from IPython.display import set_matplotlib_formats

# 4.
set_matplotlib_formats('pdf', 'svg')

<ipython-input-2-faffed60f9c3>:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


# The Function

In [ ]:
def RandomForestRegressionModel(dataset, target_variable, test_prop, min_ntrees,
                                max_ntrees, cv_folds):
  ## Inputs
  # 1. dataset: pandas DataFrame - dataset with predictor vars and output
  # 2. target_variable: character string - name of the output variable
  # 3. test_prop: float - proportion of the data to be used for testing
  # 4. min_ntrees: int - minimum number of trees to consider in the CV procedure
  # 5. max_ntrees: int - maximum number of trees to consider in the CV procedure
  # 6. cv_folds: int - number of folds to use in the CV procedure

  ## Outputs
  # 1. best_num_trees: int - optimal number of trees selected by the CV process
  # 2. mean_test_scores: list - mean cross-validated scores for each tree number
  # 3. mse_test: float - the mean squared error from the test set
  # 4. y_pred_tes: list - the optimal RFR model predicted y-values for the test set
  # 5. sorted_importances: list - feature importances sorted in descending order
  # 6. final_model: trained RFR model using the optimal number of trees
  # 7. X_train - pandas DataFrame - training set created w/ only predictors
  # 8. X_test - pandas DataFrame - testing set created w/ only predictors
  # 9. y_train - pandas DataFrame - outputs of training observations
  # 10. y_test - pandas DataFrame - outputs of test observations

  # 0. Make warnings

  # i. Warning if there are NAs in your dataset
  if dataset.isnull().values.any():
        warnings.warn("WARNING: Your dataset contains observations with missing variable values (NAs). "
                      "Consider handling missing values before proceeding with the analysis. "
                      "You can use the pandas 'fillna()' function to fill missing values with a specific value "
                      "or the 'dropna()' function to remove rows with missing values")
        return

  # ii. Warning if there are categorical variables that are not dummied out
  categorical_columns = dataset.select_dtypes(include='object').columns
  if len(categorical_columns) > 0:
    warnings.warn("WARNING: Your dataset contains columns with categorical variables that are not dummied out. "
    "Consider applying one-hot encoding or other suitable encoding techniques to handle categorical variables.")

    return

  # 1. Prepare your data and specificy predictor variables and output variable
  X = dataset.drop(target_variable, axis = 1)
  y = dataset[target_variable]

  # 2. Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size= test_prop,
                                                      random_state=42)

  # 3. Perform cross-validation to find the optimal number of trees

  # i. Create a "grid" specifying the number of trees to test
  param_grid = {'n_estimators': range(min_ntrees, max_ntrees)}

  # ii. Create the RFR model object that will be cross-validated on diff # trees
  rf_model_cv = RandomForestRegressor()

  # iii. With your RFR model, test every number of trees in the grid via CV
  grid_cv = GridSearchCV(rf_model_cv, param_grid= param_grid, cv=cv_folds)
  grid_cv.fit(X_train, y_train)

  # iv. Retrieve the cross-validated results
  cv_results = grid_cv.cv_results_
  mean_test_scores = cv_results['mean_test_score']
  std_test_scores = cv_results['std_test_score']
  num_trees = param_grid['n_estimators']

  # v. Plot the number of trees vs. cross-validation error
  plt.errorbar(num_trees, mean_test_scores, yerr= std_test_scores, marker='o',
             color = 'red', ecolor = 'blue', capsize = 3)
  plt.xlabel('Number of Trees')
  plt.ylabel('GridSearchCV CV Error Metric')
  plt.title('Number of Trees vs. Cross-Validated Error')
  plt.grid(True)
  plt.savefig('Cross-Validation-Plot.pdf',dpi = 500) # save pdf of plot
  plt.show()

  # vi. Retrieve the best model
  best_model = grid_cv.best_estimator_
  best_num_trees = grid_cv.best_params_['n_estimators']
  print("Best number of trees:", best_num_trees)

  # 5. Plot variable importance

  # i. Train the final model with the optimal number of trees
  final_model = RandomForestRegressor(n_estimators=best_num_trees)
  final_model.fit(X_train, y_train)

  # ii. Get the feature importances
  importances = final_model.feature_importances_

  # iii. Sort the feature importances in descending order
  sorted_indices = np.argsort(importances)[::-1]
  sorted_importances = importances[sorted_indices]
  sorted_features = X.columns[sorted_indices]

  # iv. Plot the variable importance
  color_palette = plt.cm.tab20.colors
  plt.barh(range(len(sorted_importances)), sorted_importances, align='center',
           color=color_palette[:len(sorted_importances)])
  plt.yticks(range(len(sorted_importances)), sorted_features)
  plt.xlabel('Feature Importance')
  plt.ylabel('Features')
  plt.title('Variable Importance')
  plt.savefig('Variable-Importance-Plot.pdf', dpi = 500) # save pdf of plot
  plt.show()

  # 6. Test the final model on the test set and calculate MSE
  y_pred_test = final_model.predict(X_test)
  rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

  return best_num_trees, mean_test_scores, rmse_test, y_pred_test, sorted_importances, final_model, X_train, X_test, y_train, y_test


# Importing Your Data

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
dataset = pd.read_csv(io.BytesIO(uploaded['filename.csv']))

# Creating the Model and Visualizing Outputs

In [ ]:
# 1. Loading dataset from pydataset and dropping NA observations
dataset = dataset.dropna()

# 2. Specifying target variable I aim to predict
target_variable = 'target_variable'

# 3. Specifying the proportion of my dataset observations to test model on
test_prop = .25

# 4. Specifying the minimum number of trees to test
min_ntrees = 5

# 5. Specifying the maximum number of trees to test
max_ntrees = 120

# 5. Specifying the number of folds to use for cross-validation
cv_folds = 10

# 6. Running function and saving outputs into variables
best_num_trees, mean_test_scores, rmse_test, y_pred_test, sorted_importances, final_model,X_train, X_test, y_train, y_test = RandomForestRegressionModel(dataset, target_variable, test_prop, min_ntrees,
                                max_ntrees, cv_folds)

In [ ]:
# 1. Prints the optimal tree number
print("Best number of trees:", best_num_trees)

# 2. Prints list of CV errors associated with each tree number tested in CV
print("Mean test scores:", mean_test_scores)

# 3. Prints list of the final model's predicted values for the test set obs
print("RFR Model Predicted Test Set Outputs:", y_pred_test)

# 4. Prints the square root of the MSE from testing the model on the test set
print("Square Root of Mean Squared Error (Test Set):", rmse_test)

# 5. Prints variable importances in descending order
print("Sorted importances:", sorted_importances)

# 6. Pints the final model object
print("Final model:", final_model)